In [10]:
#Will need to create a vector database that has all of PCIO's training material on it. This ensures the LLM can write cold outreach, but actually knows what PCIO is all about. 
#A more advanced version of this would include training the vector db on all of dad's emails. Will probably need to do vector db classes over again




# DATA SHOULD ALREADY BE IN HUBSPOT AT THIS POINT. 
#Check Hubspot and see when last outreach was. if never, then craft outreach.
#WHAT IS MATT'S SWARM METHOD?
#send 3 weeks to first lead, then if no response, send to second lead,3 weeks, then third week 

    
#GIVE THE CLIENT SOMETHING. EITHER SHOW YOU KNOW ABOUT THEIR BUSINESS, SHOW YOU KNOW THEIR PAIN POINTS, SHOW YOU KNOW SOMETHING. KEEP EMAIL SHORT SO IT DOESNT LOOK LIKE AI.

# THE TRAINING AND GENERATING A NOT SHITTY COLD EMAIL IS THE PRIMARY OBJECTIVE HERE. 

# Generate a graphic of what PCIO helpdesk could look like, and show how their setup may currently be. Show the cost savings.

#All of the dependencies are listed in the .toml file. Ensure those are going with Poetry
#ensure your .env file is in the right place 

#make sure you're using poetry shell to get all of the dependencies going in your folder




#### Example outbound. Do an Analysis of the company, amount of people, no tech people on staff, x size, probably spending around x. If that's anywhere near correct, we can save you money. 




import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

print(os.getcwd())
dotenv_path = find_dotenv()
print(f"Loading .env file from: {dotenv_path}")
_ = load_dotenv(dotenv_path)


from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import FewShotChatMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate

from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI

creativeLlmModel = OpenAI(temperature=0.9)
chatModel = ChatOpenAI(model="gpt-4o-mini")
llmModel = OpenAI()

/Users/kevincorstorphine/Desktop/Agentic_Start
Loading .env file from: /Users/kevincorstorphine/Desktop/Agentic_Start/.env


In [11]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("model_background_personality.txt")
scrape_data1 = TextLoader("scraped_pages/example_com_combined.txt")

loaded_data = loader.load()
scrape_data1 = scrape_data1.load()


In [16]:
# If you need smaller chunks, you can enforce a maximum chunk size
# to ensure that the splitter splits. It looks for logical places to split
# and the .txt scrapes are jumbled and don't have logical breaks.

# It may make sense to 


# Splitters to call small sections of relevant data, but it is also limiting in how to craft a creative response AFAIK

from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

In [17]:
load = text_splitter.create_documents([loaded_data[0].page_content])
scrape1 = text_splitter.create_documents([scrape_data1[0].page_content])

Created a chunk of size 4565, which is longer than the specified 1000
Created a chunk of size 4565, which is longer than the specified 1000
Created a chunk of size 1711, which is longer than the specified 1000
Created a chunk of size 3141, which is longer than the specified 1000
Created a chunk of size 3098, which is longer than the specified 1000
Created a chunk of size 1360, which is longer than the specified 1000
Created a chunk of size 2155, which is longer than the specified 1000
Created a chunk of size 1037, which is longer than the specified 1000
Created a chunk of size 1738, which is longer than the specified 1000
Created a chunk of size 27868, which is longer than the specified 1000
Created a chunk of size 1730, which is longer than the specified 1000
Created a chunk of size 29895, which is longer than the specified 1000


In [18]:
# len(load)
# len(scrape1)


In [19]:
#NEW LCEL


#User's Input > Prompt Template > LLM Model > Output Parser> Final Output


In [20]:
# TO DO BETTER PROMPTS READ THE E-BOOK ON PROMPTING


#I may need to use vector databases here, or maybe not. not totally sure yet.

In [23]:

#Loaded_data is the personality I gave the model in the model_background_personality.txt file. I told it to be a pirate. In the live version
#change this to the background about API3 and the product. 

#it is a pirate doing cold outreach selling bananas. 



from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a {kind_of_entity}."),
        ("human", "Hello, can you please answer a question?"),
        ("ai", "Sure!"),
        ("human", "{user_input}{scrape_detail}"),
    ]
)

messages = chat_template.format_messages(
    kind_of_entity=str(load),
   # topic="pirating",
    user_input= "Please write an outbound cold email about the problems they may be facing without a reliable helpdesk IT support service",
    scrape_detail=str(scrape1),
)

response = chatModel.invoke(messages)
print(response)

content="Subject: Are You Facing IT Support Challenges?\n\nHi [Recipient's Name],\n\nI hope this message finds you well. \n\nIn today's fast-paced business environment, having a reliable IT support service is more crucial than ever. Without it, you might be encountering challenges such as prolonged downtime, decreased productivity, and frustrated employees. Each of these issues can lead to a negative impact on your bottom line and customer satisfaction.\n\nImagine a scenario where your team faces a technical issue, and without immediate support, projects are stalled, deadlines are missed, and stress levels rise. This not only affects your operations but could also tarnish your reputation.\n\nAt PCIO, we understand the importance of seamless IT operations. We specialize in providing robust helpdesk support that addresses these challenges head-on. Our services ensure that your team can focus on their core tasks without the distraction of unresolved technical issues.\n\nI would love to di

In [22]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

#examples are a way to train the model to react exactly how you want it to.
#in the class example, this was is way to get it to speak Spanish
#FewshotChatprompttemplate is just chatPromptTemplate with examples
#Use this further modeling to refine the pitch, say waht works, what doesn't, etc.

#train AI on how you want an outbound email to look. These are just two examples of how to train it:
examples = [
    {"input": "hi!", "output": "¡hola!"},
    {"input": "bye!", "output": "¡adiós!"},
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", " you are only targeting Spanish speaking communities and are translating to spanish"),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

#a chain defines a sequence of order in which to run


chain = final_prompt | chatModel | output_parser

NameError: name 'output_parser' is not defined

In [ ]:
print(chain.invoke({"input": "please do a sales pitch to someone based on the product you sell"}))

In [ ]:
#READ PROMPTING BOOK AND DIAL THIS ALL IN

In [21]:



# FOR ADDITIONAL JSON PARSING STUFF< CHECK OUT THE PYDANTIC LIBRARY


# from langchain_core.prompts import PromptTemplate
# from langchain.output_parsers.json import SimpleJsonOutputParser

# json_prompt = PromptTemplate.from_template(
#     "Return a JSON object with an `answer` key that answers the following question: {question}"
# )

# json_parser = SimpleJsonOutputParser()

# json_chain = json_prompt | llmModel | json_parser

In [24]:
# json_parser.get_format_instructions()

In [25]:
# json_chain.invoke({"question": "What is the biggest country?"})

In [ ]:
#For opt-out list, do opt out button or whatever.